# Fetching Feature Benchmarks

In [1]:
import gc
import rmm                                                                                                                                                                                                 
import torch
import dgl
import numpy as np
import cupy as cp

#TODO: Enable in torch nightly
# torch.cuda.memory.change_current_allocator(rmm.rmm_torch_allocator)
        
import cugraph_dgl
from dgl.data import AsNodePredDataset
from dgl.dataloading import DataLoader, NeighborSampler, MultiLayerFullNeighborSampler
from ogb.nodeproppred import DglNodePropPredDataset

/datasets/vjawa/miniconda3/envs/all_cuda-115_arch-x86_64/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Create Cluter

In [2]:
single_gpu = False

In [3]:
if not single_gpu:
    from dask_cuda import LocalCUDACluster
    from dask.distributed import Client
    import cugraph.dask.comms.comms as Comms
    cluster = LocalCUDACluster(protocol='tcp',rmm_pool_size='25GB', CUDA_VISIBLE_DEVICES='1,2')
    client = Client(cluster)
    Comms.initialize(p2p=True)
else:
    rmm.reinitialize(pool_allocator=True, initial_pool_size=5e9, maximum_pool_size=20e9)
    torch.cuda.memory.change_current_allocator(rmm.rmm_torch_allocator)

2023-01-16 18:55:16,010 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-01-16 18:55:16,011 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-01-16 18:55:16,095 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-01-16 18:55:16,095 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
libibverbs: Warning: no userspace device-specific driver found for /sys/class/infiniband_verbs/uverbs1
libibverbs: Warning: no userspace device-specific driver found for /sys/class/infiniband_verbs/uverbs1
libibverbs: Warning: no userspace device-specific driver found for /sys/class/infiniband_verbs/uverbs3
libibverbs: Warning: no userspace device-specific driver found for /sys/class/infiniband_verbs/uverbs3
libibverbs: Warning: no userspace device-specific driver found for /sys/class/infiniband_verbs/uverbs0
libibverbs: Warning: no userspace device-specific driver found for /sys/class/infiniban

In [4]:
dataset = AsNodePredDataset(DglNodePropPredDataset("ogbn-products",root='/datasets/vjawa/gnn'))
## Adding Self loops to make testing easier
## As we fail with isolated edges
## in cuGraph
## See comment: https://github.com/rapidsai/cugraph/pull/2997
g = dgl.add_self_loop(dataset[0])

## Create   cugraph_dgl.cugraph_storage

In [5]:
%time cugraph_g = cugraph_dgl.cugraph_storage_from_heterograph(g, single_gpu=single_gpu)

CPU times: user 6.79 s, sys: 2.39 s, total: 9.18 s
Wall time: 11.9 s


In [6]:
indices=np.random.randint(0,cugraph_g.num_nodes(),size=20_000)
%time output = cugraph_g.get_node_storage(key='feat',ntype='_N').fetch(indices)

CPU times: user 132 ms, sys: 263 ms, total: 395 ms
Wall time: 14 ms
